In [13]:
import pandas as pd

### Import des fichiers

In [14]:
years = ["2014","2015","2016","2017"]

In [15]:
data_immo = {}
for year in years:
    tempo = pd.read_csv('../../data/foncier/immobilier_' + year + '_clean_month.csv',sep=',',dtype= {'code_commune' : str})
    tempo.rename(columns = {'valeur_fonciere' : 'valeur' },inplace=True)
    data_immo[year] = tempo

In [16]:
data_immo['2017'].sort_values(by=['id_bien','year','month']).head()

,id_bien,month,year,surface,nb_mutation,valeur
1337280,01001_1_2,1,2017,0.0,0.0,0.0
1337281,01001_1_2,2,2017,0.0,0.0,0.0
1337282,01001_1_2,3,2017,0.0,0.0,0.0
1337283,01001_1_2,4,2017,0.0,0.0,0.0
1337284,01001_1_2,5,2017,0.0,0.0,0.0


In [17]:
data_immo['2017'].shape

(1733400, 6)

### Ne garder que les biens qui sont présents sur toutes les années

on récupère d'abord les ids de biens présent sur toutes les années

In [18]:
id_bien_all_year = {}
for key, data in data_immo.items():
    
    temp = set(data["id_bien"].drop_duplicates())
    
    if len(id_bien_all_year) == 0:
        id_bien_all_year = temp
    else:
        id_bien_all_year = id_bien_all_year.intersection(temp)
    print(len(id_bien_all_year))    


127506
85124
69998
62593


on filtre ensuite les biens en fonctions de ces ids de biens

In [19]:
for key, data in data_immo.items():
    data_immo[key] = data[data.id_bien.isin(id_bien_all_year)]

### Concaténer les data frames en un seul

In [20]:
list(data_immo.values())

[           id_bien  month  year     surface  nb_mutation         valeur
 0        93072_2_4      1  2014   70.000000          5.0  179202.804000
 1        93072_2_4      2  2014   72.666667          3.0  204337.850000
 2        93072_2_4      3  2014   74.500000          2.0  142500.000000
 3        93072_2_4      4  2014   79.000000          3.0  193708.880000
 4        93072_2_4      5  2014   68.750000          4.0  169725.465000
 5        93072_2_4      6  2014   83.000000          1.0  160000.000000
 6        93072_2_4      7  2014   73.375000          8.0  208857.476250
 7        93072_2_4      8  2014   75.500000          2.0  171500.000000
 8        93072_2_4      9  2014   77.500000          2.0  184500.000000
 9        93072_2_4     10  2014   81.200000          5.0  213588.786000
 10       93072_2_4     11  2014   72.000000          1.0  180000.000000
 11       93072_2_4     12  2014   75.500000          2.0  212500.000000
 12       93001_2_2      1  2014   41.764706       

In [21]:
immo_concat = pd.concat(list(data_immo.values()))

In [22]:
immo_concat.shape

(3004464, 6)

### Filtre des lignes selon le nombre de mutation ( on vire tous les types de biens avec moins de 10 ventes sur 2014 - 2017 )

On va virer toutes les types de biens qui totalisent moins de 10 mutations sur toutes les années

In [23]:
sum_by_id_bien = immo_concat.groupby('id_bien').sum().reset_index()

In [24]:
sum_by_id_bien.head()

,id_bien,month,year,surface,nb_mutation,valeur
0,01001_1_3,312,96744,399.0,5.0,9.123900e+05
1,01001_1_4,312,96744,923.0,13.0,1.757733e+06
2,01004_1_1,312,96744,428.5,11.0,8.456350e+05
3,01004_1_2,312,96744,1022.0,24.0,1.825696e+06
4,01004_1_3,312,96744,2744.5,66.0,5.612306e+06


In [25]:
sum_by_id_bien.shape

(62593, 6)

sum_by_id_bien = sum_by_id_bien[sum_by_id_bien.nb_mutation > 100]

In [26]:
id_bien_to_keep = sum_by_id_bien.id_bien

In [27]:
immo_concat.shape

(3004464, 6)

In [28]:
immo_concat = immo_concat[immo_concat.id_bien.isin(id_bien_to_keep)]
immo_concat.shape

(3004464, 6)

Cela vire 969312 lignes

In [29]:
immo_concat.head()

,id_bien,month,year,surface,nb_mutation,valeur
0,93072_2_4,1,2014,70.000000,5.0,179202.804
1,93072_2_4,2,2014,72.666667,3.0,204337.850
2,93072_2_4,3,2014,74.500000,2.0,142500.000
3,93072_2_4,4,2014,79.000000,3.0,193708.880
4,93072_2_4,5,2014,68.750000,4.0,169725.465


In [30]:
immo_concat.shape

(3004464, 6)

In [34]:
immo_concat.head()

,id_bien,month,year,surface,nb_mutation,valeur
0,93072_2_4,1,2014,70.000000,5.0,179202.804
1,93072_2_4,2,2014,72.666667,3.0,204337.850
2,93072_2_4,3,2014,74.500000,2.0,142500.000
3,93072_2_4,4,2014,79.000000,3.0,193708.880
4,93072_2_4,5,2014,68.750000,4.0,169725.465


### On enregistre comme csv

In [35]:
immo_concat.to_csv('../../data/join/immo_shiny.csv',index=False)

In [59]:
bien_table.to_csv('../../data/table/type_bien_immo.csv',index=False)